In [1]:
import pandas as pd
import requests
import sys
import json

# import sys then find the folder where the components is located and add a path to it.
# to be able to run test.ipynb inside components
# sys.path.append(r"D:\test_python_project")

from components.api_request import ApiRequest
from components.fetch_data import FetchData
# from components.retrieve_data import RetrieveData

In [3]:
fields = 'id,name,adaccounts{id,name,currency,amount_spent,campaigns{name,insights{spend,clicks,impressions,cpc,cpm,ctr,actions},daily_budget,budget_rebalance_flag,effective_status,start_time,stop_time,adsets{insights{adset_id,adset_name,date_start,date_stop},ads{name,adcreatives{body},creative{video_id{thumbnails}}},targeting{geo_locations,genders,age_min,age_max,flexible_spec{interests{id,name}}},placement,destination_type,promoted_object{pixel_id}}}}'

df = pd.read_csv(r"D:\python_project\components\facebook_tokens.csv")
df.set_index("Profile ID", inplace = True)

In [4]:
dictionary = df["Access Token"].to_dict()

In [5]:
def get_request(url):
    response = requests.get(url, timeout = 40)
    return response.json()

def get_data_of_campaigns(adaccounts_data, key1 = "campaigns"):
    list_that_have_campaigns = []

    for campaigns in adaccounts_data:
        if key1 in campaigns.keys():
            list_that_have_campaigns.append(campaigns[key1])
    
    return list_that_have_campaigns

def get_index_position_and_data_if_next_is_found(list_of_campaign_datas: list[dict], key = "paging") -> dict:
    index_and_data = {}
    
    for idx, element in enumerate(list_of_campaign_datas):
        target = element[key]

        if "next" in target:
            url = target["next"]
            json = get_request(url)
            index_and_data[idx] = json
    
    return index_and_data

def extract_next_url_if_next_keyword_is_found_in_the_dictionary_of_datas_and_index(index_and_datas: dict) -> dict:
    new_data = {}

    for key, value in index_and_datas.items():
        if "next" in value["paging"]:
            url_link = value["paging"]["next"]
            json_data = get_request(url_link)
            new_data[key] = json_data["data"]

    return new_data

def overwrite_datas_after_extracted_next_url(original_dict: dict, process_dict: dict) -> dict:
    for key in original_dict.keys():

        for index_pos, data in process_dict.items():
            if index_pos == key:
                original_dict[index_pos]["data"].extend(data)
    
    return original_dict

def delete_pagings_when_done_extracting(index_and_datas: dict) -> None:
    new_index_and_datas = {}

    for key, value in index_and_datas.items():
        value_copy = value.copy()
        del value_copy["paging"]

        new_index_and_datas[key] = value_copy

    return new_index_and_datas    

In [6]:
all_data, datas_with_campaigns = [], []

for ids, token in dictionary.items():
    get_response = ApiRequest(profile_id= ids, access_token = token)
    get_response.execute_response()    

    if get_response.data is not None and len(get_response.data) != 0:
        all_data.extend(get_response.data)
        if "adaccounts" in get_response.data[0]:
            adaccounts_data = get_response[0]["adaccounts"]["data"] # added __getitem__

            campaigns = get_data_of_campaigns(adaccounts_data)
            datas_with_campaigns.extend(campaigns)

In [7]:
copy_datas_with_campaigns = datas_with_campaigns.copy()
all_data_copy = all_data.copy()

In [8]:
# original data
index_and_datas = get_index_position_and_data_if_next_is_found(copy_datas_with_campaigns)

In [9]:
# process data that have next keyword
next_in_next = extract_next_url_if_next_keyword_is_found_in_the_dictionary_of_datas_and_index(index_and_datas)

In [10]:
# new data
result = overwrite_datas_after_extracted_next_url(index_and_datas, next_in_next)

In [11]:
# deleting the next
final_result = delete_pagings_when_done_extracting(result)

In [12]:
my_list = [value["data"] for value in final_result.values()]

In [13]:
all_data_adaccounts_campaigns_with_next_keyword = []
for element in all_data_copy:
    for idx, key in enumerate(element["adaccounts"]["data"]):
        if "campaigns" in key:
            target = element["adaccounts"]["data"][idx]["campaigns"]["paging"]
            if "next" in target:
                x_mark = element["adaccounts"]["data"][idx]["campaigns"]["data"]
                all_data_adaccounts_campaigns_with_next_keyword.append(x_mark)
                # copy_of_x_mark = x_mark.copy()
                # all_data_adaccounts_campaigns_with_next_keyword.append(copy_of_x_mark)

In [14]:
combined = [datas1 + datas2 for datas1, datas2 in zip(all_data_adaccounts_campaigns_with_next_keyword, my_list)]

In [15]:
counter = 0

for element in all_data_copy:
    for idx, key in enumerate(element["adaccounts"]["data"]):
        if "campaigns" in key:
            target = element["adaccounts"]["data"][idx]["campaigns"]["paging"]
            if "next" in target:
                element["adaccounts"]["data"][idx]["campaigns"]["data"] = combined[counter]
                counter += 1
                
                if counter >= len(combined):
                    break
    if counter >= len(combined):
        break

In [18]:
# target_dict = all_data_copy[0]["adaccounts"]["data"]
# deleting
for element in all_data_copy:
    for idx, key in enumerate(element["adaccounts"]["data"]):
        if "campaigns" in key:
            del element["adaccounts"]["data"][idx]["campaigns"]["paging"]
        

In [19]:
all_data_copy

[{'id': '241197038556518',
  'name': 'Sunny Gordon',
  'adaccounts': {'data': [{'id': 'act_424962726144378',
     'name': 'Sunny Gordon',
     'currency': 'HKD',
     'amount_spent': '0'},
    {'id': 'act_741377553612666',
     'name': 'SBJ03 | (MB | CA)',
     'currency': 'USD',
     'amount_spent': '47068',
     'campaigns': {'data': [{'name': 'AN3 | YFAWC [CBO]',
        'daily_budget': '20000',
        'budget_rebalance_flag': False,
        'effective_status': 'PAUSED',
        'start_time': '2022-10-21T06:24:50-0700',
        'adsets': {'data': [{'ads': {'data': [{'name': 'AN3-1@1 | | | V57-M1-S2 | T2',
              'adcreatives': {'data': [{'id': '23851524727260113'}]},
              'creative': {'id': '23851524727260113'},
              'id': '23851524507240113'}],
            'paging': {'cursors': {'before': 'QVFIUlU1allnVzVxaGxtWUlqVzNuUTViUWlxNE90RjdPeUtHdzNOeTJKQXRyVHBXQ0dFczh5MDhSdUZA2TkRkbmM3dDJtUUVEZAFFqaGxHN2VIQ0RCZAURqWWVB',
              'after': 'QVFIUlU1allnVzVxaGx

In [80]:
with open("test5.json", "w") as file:
    file.write(json.dumps(all_data_copy, indent = 2))